In [1]:
from dotenv import load_dotenv
import os
import boto3
import csv

load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = os.getenv("AWS_DEFAULT_REGION")

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_default_region
)

client = boto3.client('cognito-idp', region_name='eu-west-3') 

user_pool_id = 'eu-west-3_wclKmH4UE' 

pagination_token = None
total_users = 0  # Contador total de usuarios
users_with_email = 0  # Contador de usuarios con email
users_without_email = 0  # Contador de usuarios sin email

with open('usuarios_cognito.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Encabezados del archivo CSV
    writer.writerow(['Email', 'Total Usuarios sin Email', 'Total Usuarios con Email'])

    while True:
        if pagination_token:
            response = client.list_users(
                UserPoolId=user_pool_id,
                AttributesToGet=['email'],
                PaginationToken=pagination_token
            )
        else:
            response = client.list_users(
                UserPoolId=user_pool_id,
                AttributesToGet=['email']
            )

        for user in response['Users']:
            total_users += 1  # Contamos el usuario
            found_email = False  # Variable para controlar si encontramos un email

            for attribute in user['Attributes']:
                if attribute['Name'] == 'email':
                    writer.writerow([attribute['Value'], '', ''])  # Escribir email
                    users_with_email += 1
                    found_email = True
                    break

            if not found_email:
                users_without_email += 1

        pagination_token = response.get('PaginationToken')

        if not pagination_token:
            break

    # Agregar fila final con los totales
    writer.writerow(['', users_without_email, users_with_email])

print(f"Total usuarios procesados: {total_users}")
print(f"Usuarios con email: {users_with_email}")
print(f"Usuarios sin email: {users_without_email}")
print("Los correos electrónicos se han guardado correctamente en 'usuarios_cognito.csv'.")


Total usuarios procesados: 12813
Usuarios con email: 11647
Usuarios sin email: 1166
Los correos electrónicos se han guardado correctamente en 'usuarios_cognito.csv'.
